<a href="https://colab.research.google.com/github/Anuragpandey2005/lognormal-sales-analysis/blob/main/lognormalpynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#import dataset
df = pd.read_excel("/content/Sample B2C Dataset EDA.xlsx")

In [ ]:
#print dataset
df

In [ ]:
#print info of the dataset
print(df.info())


🧹 Step 2: Data Cleaning & Preparation


In [ ]:
# Assume 1 EUR = 30 TRY
try_columns = ['Price', 'Competitor Price']
conversion_rate = 30

2.2 Handle Missing Data

In [ ]:
# Print original column names for reference
print("Original Columns:")
print(df.columns.tolist())

# Clean column names
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print("\nCleaned Columns:")
print(df.columns.tolist())


In [ ]:
# Fill missing Advertising Spend with median
df['Advertising_Spend_EUR'].fillna(df['Advertising_Spend_EUR'].median(), inplace=True)


In [ ]:
print(df.isnull().sum())


In [ ]:
# Fill missing values
df['Sales_Volume'].fillna(df['Sales_Volume'].median(), inplace=True)
df['Customer_Reviews'].fillna(df['Customer_Reviews'].median(), inplace=True)

# Check if anything remains missing
print(df.isnull().sum())


Step 4: standarise the columns

* Currency Conversion (TRY → EUR)


In [ ]:
# Set exchange rate
exchange_rate = 45

# Identify rows in TRY
try_mask = df['Pricing_Currency'] == 'TRY'

# Convert relevant columns from TRY to EUR
df.loc[try_mask, ['Price', 'Competitor_Price']] = df.loc[try_mask, ['Price', 'Competitor_Price']] / exchange_rate

# Optional: update currency column to EUR after conversion
df.loc[try_mask, 'Pricing_Currency'] = 'EUR'

# Confirm conversion
print(df[try_mask][['Product_Name', 'Price', 'Competitor_Price', 'Pricing_Currency']].head())


✅ Step 5: Handle Outliers



In [ ]:
# Replace zero or negative prices with median of valid prices
valid_price_median = df[df['Price'] > 0]['Price'].median()
df.loc[df['Price'] <= 0, 'Price'] = valid_price_median

valid_comp_price_median = df[df['Competitor_Price'] > 0]['Competitor_Price'].median()
df.loc[df['Competitor_Price'] <= 0, 'Competitor_Price'] = valid_comp_price_median


In [ ]:
# Check if any products had sales but stock was 0
stock_issue = df[(df['Warehouse_Stock_Level'] == 0) & (df['Sales_Volume'] > 0)]

# Show a few for inspection
print(stock_issue[['Product_Name', 'Sales_Volume', 'Warehouse_Stock_Level']])


Step 6: Feature Engineering

In [ ]:
# 1. Price Differential: Difference from competitor
df['Price_Differential'] = df['Price'] - df['Competitor_Price']

# 2. Ad Efficiency: Sales per euro spent
df['Ad_Efficiency'] = df['Sales_Volume'] / (df['Advertising_Spend_EUR'] + 1)  # +1 avoids division by zero

# 3. Is_Stockout: Flag zero stock
df['Is_Stockout'] = df['Warehouse_Stock_Level'].apply(lambda x: 1 if x == 0 else 0)

# 4. Price Tier: Categorize into Low, Medium, High price products
df['Price_Tier'] = pd.qcut(df['Price'], q=3, labels=['Low', 'Medium', 'High'])


In [ ]:
# Check result
print(df[['Product_Name', 'Price', 'Competitor_Price', 'Price_Differential', 'Ad_Efficiency', 'Is_Stockout', 'Price_Tier']].head())


Step 7: Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set(style='whitegrid')

# 1. Total Sales Volume by Category
category_sales = df.groupby('Category')['Sales_Volume'].sum().reset_index()
sns.barplot(data=category_sales, x='Category', y='Sales_Volume', palette='Set2')
plt.title('Total Sales Volume by Category')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 2. Total Sales Volume by Price Tier
price_tier_sales = df.groupby('Price_Tier')['Sales_Volume'].sum().reset_index()
sns.barplot(data=price_tier_sales, x='Price_Tier', y='Sales_Volume', palette='Set1')
plt.title('Total Sales Volume by Price Tier')
plt.show()

Step 8: Correlation & Trend Detection


Pearson Correlation Heatmap

In [ ]:
# Correlation matrix for numeric variables
correlation_matrix = df[['Sales_Volume', 'Price', 'Competitor_Price',
                         'Price_Differential', 'Advertising_Spend_EUR',
                         'Customer_Reviews', 'Weather_Index',
                         'Social_Media_Mentions']].corr()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()


Regression Analysis (Sales vs. Drivers)

In [ ]:
import statsmodels.api as sm

# Define features and target
X = df[['Price', 'Competitor_Price', 'Advertising_Spend_EUR']]
X = sm.add_constant(X)  # Adds intercept
y = df['Sales_Volume']

# Fit model
model = sm.OLS(y, X).fit()

# View results
print(model.summary())


 Step 9: Competitor Pricing Impact (Visual Insight)


In [ ]:
# Scatter plot: Price Differential vs Sales Volume
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df, x='Price_Differential', y='Sales_Volume', hue='Category', palette='Set2')
plt.axvline(0, color='red', linestyle='--', label='Price Match')
plt.title('Price Differential vs Sales Volume')
plt.legend()
plt.show()


Step 10: Additional Insights — External Drivers

analyze:

📢 Ad Spend effectiveness

⭐ Customer Review impact

📱 Social Media buzz

☀️ Weather effect



1. Ad Spend vs Sales Volume


In [ ]:
sns.scatterplot(data=df, x='Advertising_Spend_EUR', y='Sales_Volume', hue='Category')
plt.title('Ad Spend vs Sales Volume')
plt.show()


🔹 2. Customer Reviews vs Sales Volume



In [ ]:
sns.boxplot(data=df, x='Category', y='Customer_Reviews')
plt.title('Customer Reviews by Category')
plt.show()


3. Social Media Mentions vs Sales Volume

In [ ]:
sns.lmplot(data=df, x='Social_Media_Mentions', y='Sales_Volume', hue='Category', aspect=1.5)
plt.title('Social Media Mentions vs Sales Volume')


4. Weather Index vs Sales Volume

In [ ]:
sns.lmplot(data=df, x='Weather_Index', y='Sales_Volume', hue='Category', aspect=1.5)
plt.title('Weather Index vs Sales Volume')


Step 11: Final Recommendations
Here are 5 data-driven suggestions you can include in your final slide:



🎯 Recommendations (for Slide 7)

Price Strategy:


Most products are priced close to competitors.


Recommend selectively undercutting in Electronics to gain volume.


Use price optimization for Home Goods (less price-sensitive).

Ad Spend:

Optimize, don’t increase — high ad spend ≠ higher sales.

Focus on high ROI products based on Ad_Efficiency.

Review Management:

Improve product listings for low-rated items to boost perception.

Prioritize high-rated products in promotions.


Stock Management:


Ensure stock availability for top-selling items (many stockouts had high sales).

Consider predictive inventory planning.


Social & Weather:


Weather influence is minimal but noticeable — especially for seasonal clothing.

Social buzz can be tracked for post-sale engagement but isn't a strong driver.

